In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [2]:
import yaml
import torch
from torch.nn.functional import mse_loss
from torchvision.transforms.functional import to_pil_image, to_tensor
from diffusers import AutoencoderKL, StableDiffusionImg2ImgPipeline
from PIL import Image

/root/miniconda3/envs/gaussian-editor/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = "stabilityai/stable-diffusion-2-1"
diffusion_pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

Loading pipeline components...: 100%|██████████| 6/6 [00:04<00:00,  1.41it/s]


In [4]:
# get list of all images in src/outputs folder
images = [Image.open(f"outputs/{f}") for f in os.listdir("outputs") if f.endswith(".png")]

In [ ]:
img = images[0]

In [5]:
results = {}
generator = torch.Generator().manual_seed(42)
for img in images:
    reconstructed = diffusion_pipeline(prompt='', image=img, strength=0.5, generator=generator, output_type='pt').images[0]
    padded = torch.nn.functional.pad(reconstructed, (0,0,1,3), "constant", 0)
    original_img = to_tensor(img).to("cuda")
    mse = mse_loss(padded, original_img)
    results[img.filename.split('/')[1]] = mse.item()

100%|██████████| 25/25 [00:05<00:00,  4.64it/s]


In [6]:
results

{'rendering_1.0_0.png': 0.02181563898921013,
 'rendering_1.0_1.png': 0.016872664913535118,
 'rendering_1.0_2.png': 0.016252512112259865,
 'rendering_1.0_3.png': 0.01090245135128498,
 'rendering_1.0_4.png': 0.01635332964360714,
 'rendering_1.1_0.png': 0.022132404148578644,
 'rendering_1.1_1.png': 0.017927592620253563,
 'rendering_1.1_2.png': 0.016170838847756386,
 'rendering_1.1_3.png': 0.011726079508662224,
 'rendering_1.1_4.png': 0.016623998060822487,
 'rendering_1.2_0.png': 0.022527087479829788,
 'rendering_1.2_1.png': 0.017826566472649574,
 'rendering_1.2_2.png': 0.01602393016219139,
 'rendering_1.2_3.png': 0.011837710626423359,
 'rendering_1.2_4.png': 0.0163355004042387,
 'rendering_1.3_0.png': 0.022044001147150993,
 'rendering_1.3_1.png': 0.016196241602301598,
 'rendering_1.3_2.png': 0.016354553401470184,
 'rendering_1.3_3.png': 0.011651843786239624,
 'rendering_1.3_4.png': 0.016746925190091133,
 'rendering_1.4_0.png': 0.022482460364699364,
 'rendering_1.4_1.png': 0.01656924746930

In [7]:
# save results to yaml file
# Define the path to the YAML file
yaml_file = "results_same_generator.yaml"

# Save the results dictionary to the YAML file
with open(yaml_file, "w") as file:
    yaml.dump(results, file)



In [ ]:
# open results file
with open("results.yaml", "r") as file:
    results = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
results

In [ ]:
grouped_results = {}
for key in results.keys():
    depth_scale = key.split("_")[1]
    if depth_scale not in grouped_results:
        grouped_results[depth_scale] = []
    grouped_results[depth_scale].append((int(key.split("_")[2].split(".")[0]), results[key]))
grouped_results

In [ ]:
# get the average mse for each depth scale
average_results = {}
for key in grouped_results.keys():
    average_results[key] = sum([x[1] for x in grouped_results[key]]) / len(grouped_results[key])
average_results

In [ ]:
# get min max from average results
min_key = min(average_results, key=average_results.get)
max_key = max(average_results, key=average_results.get)
min_key, average_results[min_key], max_key, average_results[max_key]

In [ ]:
# filter out the depth_scales between 1.0 and 1.6 and compute the min max from average results
filtered_results = {key: value for key, value in average_results.items() if not 1.0 <= float(key) <= 1.6}
min_key = min(filtered_results, key=filtered_results.get)
max_key = max(filtered_results, key=filtered_results.get)
min_key, filtered_results[min_key], max_key, filtered_results[max_key]


In [ ]:
new_results = {}
views = range(5)
for view in views:
    view_results = {depth: views_mse[view][1] for depth, views_mse in grouped_results.items()}
    filtered_view_results = {key: value for key, value in view_results.items() if not 1.0 <= float(key) <= 1.6}
    min_key = min(filtered_view_results, key=filtered_view_results.get)
    max_key = max(filtered_view_results, key=filtered_view_results.get)
    min_key, filtered_view_results[min_key], max_key, filtered_view_results[max_key]
    new_results[view] = {"min": (min_key, filtered_view_results[min_key]), "max": (max_key, filtered_view_results[max_key])}
new_results
# view_results = {depth: views_mse[view][1] for depth, views_mse in grouped_results.items()}
# view_results

In [ ]:
# get min max mse for view 0 (filter out the depth_scales between 1.0 and 1.6)
filtered_view_results = {key: value for key, value in view_results.items() if not 1.0 <= float(key) <= 1.6}
min_key = min(filtered_view_results, key=filtered_view_results.get)
max_key = max(filtered_view_results, key=filtered_view_results.get)
min_key, filtered_view_results[min_key], max_key, filtered_view_results[max_key]